In [1]:
import sys
import os
import yaml

sys.path.append('/Users/raphaelravinet/Code/algo_trading')
sys.path.append('/Users/raphaelravinet/Code/Fin_Database')

import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import ta
from log_config import setup_logging
from Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials
from Pre_Processing.pre_processing import PreProcessing
from Feature_Engineering.feature_engineering import TechnicalIndicators
from Feature_Engineering.vol_prediction import IntradayVol
from pipeline import Pipeline

from sklearn.metrics import roc_auc_score, roc_curve, auc

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, log_loss, confusion_matrix, ConfusionMatrixDisplay

In [2]:
tickers = ['AAPL','MSFT']

In [3]:
pipe = Pipeline(tickers)

In [4]:
daily_data = pipe.pipeline()

In [9]:
daily_data['AAPL'].columns

Index(['id', 'date', 'timestamp', 'ticker', 'open', 'high', 'low', 'close',
       'volume', 'vwap', 'transactions', 'RSI_14', 'RSI_2', 'MACD', 'log_ret',
       'ROC', 'Stoch', 'ADX', 'ADX_pos', 'ADX_neg', 'rolling_H-L_25', 'ATR',
       'IBS', 'OBV', 'sma_5', 'ema_5', 'sma_10', 'ema_10', 'sma_30', 'ema_30',
       'sma_50', 'ema_50', 'sma_100', 'ema_100', 'sma_200', 'ema_200',
       'sma_300', 'ema_300'],
      dtype='object')

In [7]:
daily_data['MSFT']

,id,date,timestamp,ticker,open,high,low,close,volume,vwap,...,sma_30,ema_30,sma_50,ema_50,sma_100,ema_100,sma_200,ema_200,sma_300,ema_300
date,,,,,,,,,,,,,,,,,,,,,
2003-09-10,300779,2003-09-10,1063166400000,MSFT,28.030,28.18,27.4800,27.55,54529513.0,27.8702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-09-11,300780,2003-09-11,1063252800000,MSFT,27.660,28.11,27.5900,27.84,37921893.0,27.8471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-09-12,300781,2003-09-12,1063339200000,MSFT,27.480,28.40,26.7000,28.34,55655033.0,27.8832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-09-15,300782,2003-09-15,1063598400000,MSFT,28.370,28.61,28.3300,28.36,41445449.0,28.4527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-09-16,300783,2003-09-16,1063684800000,MSFT,28.410,28.95,28.3200,28.90,51687185.0,28.6075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-05,306039,2024-08-05,1722830400000,MSFT,389.170,401.04,385.5800,395.15,38805980.0,394.8028,...,442.293000,432.532527,437.8632,432.975567,426.8981,425.012538,404.13535,403.265123,379.687517,383.585870
2024-08-06,306040,2024-08-06,1722916800000,MSFT,400.000,405.67,398.5000,399.61,23608739.0,401.6113,...,440.691000,430.408493,437.3154,431.667114,426.7432,424.509517,404.48285,403.228754,379.933150,383.692343
2024-08-07,306041,2024-08-07,1723003200000,MSFT,408.635,410.08,397.4700,398.43,19206613.0,402.7799,...,438.940333,428.345364,436.6808,430.363698,426.4753,423.993091,404.81840,403.181005,380.151617,383.790267


In [ ]:
import plotly.graph_objects as go


In [ ]:
aapl_min.drop(columns = 'IBS', inplace=True)

In [ ]:
aapl_min

In [ ]:
data = aapl_min.copy()

In [ ]:
data

In [ ]:
data = aapl_min.dropna(inplace=True)

In [ ]:
data.index = pd.to_datetime(data.index)

In [ ]:
fig = go.Figure(data = go.Candlestick(x = data.index,
                                      open = data['open'],
                                      high = data['high'],
                                      low = data['low'],
                                      close = data['close'],
                                      increasing=dict(line=dict(color='black')),
                                      decreasing=dict(line=dict(color='red')),
                                      showlegend=False))

layout = go.Layout(
    title=' Apple Stock Price - 2020 to 2024',
    yaxis=dict(title='Price (USD)'),
    xaxis=dict(title='Date'),
    template = 'ggplot2',
    xaxis_rangeslider_visible=False,
    yaxis_gridcolor='white',
    xaxis_gridcolor='white',
    yaxis_tickfont=dict(color='black'),
    xaxis_tickfont=dict(color='black'),
    margin=dict(t=50,l=50,r=50,b=50)
)

fig.update_layout(layout)


fig.show()

In [ ]:
data['target'] = np.where(data['log_ret'].shift(-1) > 0, 1, 0)


In [ ]:
train = data.iloc[:int(0.8*len(data))]
test = data.iloc[int(0.8*len(data)):]
X_train = train.drop(columns = ['target', 'ticker','date.1'])
y_train = train['target']
X_test = test.drop(columns = ['target', 'ticker','date.1'])
y_test = test['target']

In [ ]:
models = [
    ("XGBoost", XGBClassifier()),
    ("CatBoost", CatBoostClassifier(verbose=False)),
    ("RandomForest", RandomForestClassifier())
]

In [ ]:
plt.figure(figsize=(10, 10))
for name, model in models:
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1] 
    y_pred = model.predict(X_test) 
    
    # Metrics
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    log_loss_value = log_loss(y_test, y_pred_proba)

    print(f'{name} ROC AUC: {roc_auc}')
    print(f'{name} Log Loss: {log_loss_value}')
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    
    #CM
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'{name} Confusion Matrix')
    plt.show()